# Introduction

TODO


# Imports

In [1]:
from __future__ import print_function
import tensorflow as tf
import keras.backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate, GRU, Dot, TimeDistributed, Activation, Embedding, Lambda, Concatenate, Reshape
from keras import optimizers
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import json
from keras.models import load_model
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
import re
%matplotlib inline

Using TensorFlow backend.


# Utility functions

In [2]:
# Limit gpu allocation. allow_growth, or gpu_fraction
def gpu_alloc():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

In [3]:
gpu_alloc()

In [4]:
def build_chars_vocab(all_texts):
    '''Build vocab dictionary to victorize chars into ints'''
    vocab_to_int = {}
    count = 0 # Start index for any char will be 1, as 0 is masked by the Embedding/Masking layer
    codes = ['UNK', ' ', '\t','\n']# Start 'UNK' at the first entry, to keep its index=0 to be masked
    for code in codes:
        if code not in vocab_to_int:
            vocab_to_int[code] = count
            count += 1    
    
    for sentence in all_texts:
        for char in sentence:
            if char not in vocab_to_int:
                vocab_to_int[char] = count
                count += 1


    '''''Build inverse translation from int to char'''
    int_to_vocab = {}
    for character, value in vocab_to_int.items():
        int_to_vocab[value] = character
        
    return vocab_to_int, int_to_vocab

In [5]:
def vectorize_sentences_data(input_texts, target_labels, max_sents_per_doc, max_words_per_sent, max_chars_per_word, 
                             num_classes, char2int):

    
    hier_input_data_lst = []
    hier_target_data_lst = []

    if(target_labels == None):
        target_labels = np.zeros(len(input_texts), dtype='int32')
    
    for _, (input_text, target_label) in enumerate(zip(input_texts, target_labels)):
        hier_input_data = np.zeros(max_sents_per_doc, 
                                    max_words_per_sent, 
                                    max_chars_per_word), dtype='float32')


        hier_target_data = np.zeros(num_classes), dtype='float32')

        #sents_lst = sent_tokenize(clean_str(BeautifulSoup(input_text).get_text())) # TODO: Move to clean str
        sents_lst = sent_tokenize(input_text)
        
        
        if len(sents_lst) > max_sents_per_doc:
            continue
        j=0
        for _, sent in enumerate(sents_lst):
                
            words_lst = word_tokenize(input_text)
            
            if(len(words_lst) > max_words_per_sent):
                continue
            
            
            k=0
            for _, word in enumerate(words_lst):
                
                
                if(len(word) > max_chars_per_word):
                    continue
                
                
                l=0
                for l, char in enumerate(word):
                    # c0..cn
                    if(char in char2int):
                        hier_input_data[j, k, l] = char2int[char]
                        try:
                            hier_target_data[target_label] = 1
                        except:
                            print(target_label)
                        l=l+1
                k=k+1
            j=j+1
                            
        hier_input_data_lst.append(hier_input_data)
        hier_target_data.append(hier_target_data)
                
    return np.array(hier_input_data), np.array(hier_target_data)

SyntaxError: invalid syntax (<ipython-input-5-60098521a3e0>, line 14)

In [ ]:
def build_chars2word_model_simple_BiLSTM(num_encoder_tokens, latent_dim):
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(None,), dtype='float32')
    encoder_inputs_ = Embedding(num_encoder_tokens, latent_dim,                                                       
                            mask_zero=True)(encoder_inputs)    
    #encoder_inputs = Input(shape=(None, num_encoder_tokens))
    encoder = Bidirectional(LSTM(latent_dim, return_state=True, return_sequences=True)) # Bi LSTM
    encoder_outputs, state_f_h, state_f_c, state_b_h, state_b_c = encoder(encoder_inputs_)# Bi LSTM
 
    encoder_embedding_output = Lambda(lambda x: x[:,-1,:])(encoder_outputs)
    encoder_word_embedding_model = Model(input=encoder_inputs, output=encoder_embedding_output)

    return encoder_word_embedding_model

In [ ]:
def build_words2sent_model_simple_BiLSTM(encoder_word_embedding_model, 
                           max_words_seq_len, 
                           max_char_seq_len, 
                           latent_dim):
    # Define an input sequence and process it.

    inputs = Input(shape=(max_words_seq_len, max_char_seq_len,), dtype='float32')
    #print(inputs.shape)
    input_words = TimeDistributed(encoder_word_embedding_model)(inputs)

    encoder_inputs_ = input_words   
    #encoder_inputs = Input(shape=(None, char_vocab_size))
    encoder = Bidirectional(LSTM(latent_dim, return_state=True, return_sequences=True)) # Bi LSTM
    encoder_outputs, state_f_h, state_f_c, state_b_h, state_b_c = encoder(encoder_inputs_)# Bi LSTM
        
    encoder_embedding_output = Lambda(lambda x: x[:,-1,:])(encoder_outputs)
    encoder_sentence_embedding_model = Model(input=inputs, output=encoder_embedding_output)

    return encoder_sentence_embedding_model

In [ ]:
 
def build_sent2doc_model(encoder_sentence_embedding_model, 
                         max_sents_seq_len, 
                         max_words_seq_len, 
                         max_char_seq_len, 
                         word2sent_latent_dim,
                         sent2doc_latent_dim):
    
    inputs = Input(shape=(max_sents_seq_len, max_words_seq_len, max_char_seq_len,), dtype='float32')
    
    sents_states = []
    
    for s in range(max_sents_seq_len):
        
        encoder_words_inputs = Lambda(lambda x: x[:,s,:,:])(inputs)
        #print(encoder_words_inputs.shape)
        encoder_words_outputs = encoder_sentence_embedding_model(encoder_words_inputs)
        encoder_words_outputs = Reshape((1,word2sent_latent_dim*2))(encoder_words_outputs)
        #_, h, c = encoder_sentence_embedding_model(encoder_words_inputs)
        '''
        input_words = TimeDistributed(encoder_word_embedding_model)(inputs)

        encoder_inputs_ = input_words   
        #encoder_inputs = Input(shape=(None, char_vocab_size))
        encoder = Bidirectional(LSTM(latent_dim, return_state=True, return_sequences=True)) # Bi LSTM
        encoder_outputs, state_f_h, state_f_c, state_b_h, state_b_c = encoder(encoder_inputs_)# Bi LSTM

        encoder_embedding_output = Lambda(lambda x: x[:,-1,:])(encoder_outputs)
        '''
        
        
        #encoder_words_states = Concatenate()([h,c])
        #print(encoder_chars_states)
        #encoder_words_states = Reshape((1,word2sent_latent_dim*4))(encoder_words_states)
        #print(encoder_words_outputs.shape)
        sents_states.append(encoder_words_outputs)
    print(sents_states[0]._keras_shape)
    input_sents = Concatenate(axis=-2)(sents_states)
    #print(input_sents.shape)
    encoder_inputs_ = input_sents   
    encoder = Bidirectional(LSTM(sent2doc_latent_dim, return_state=True, return_sequences=True)) # Bi LSTM
    encoder_outputs, state_f_h, state_f_c, state_b_h, state_b_c = encoder(encoder_inputs_)# Bi LSTM
    state_h = Concatenate()([state_f_h, state_b_h])# Bi LSTM
    state_c = Concatenate()([state_f_c, state_b_c])# Bi LSTM
    encoder_embedding_output = Lambda(lambda x: x[:,-1,:])(encoder_outputs)
    
    encoder_document_embedding_model = Model(input=inputs, output=encoder_embedding_output)
    '''
    preds = Dense(2, activation='softmax')(encoder_embedding_output)
    model = Model(inputs, preds)
    '''
    #return model, encoder_document_embedding_model
    return encoder_document_embedding_model

In [ ]:
def build_hier_senti_model(encoder_document_embedding_model,
                           max_sents_seq_len, 
                           max_words_seq_len, 
                           max_char_seq_len):
    inputs = Input(shape=(max_sents_seq_len, max_words_seq_len, max_char_seq_len,), dtype='float32')
    encoder_embedding_output = encoder_document_embedding_model(inputs)
    preds = Dense(2, activation='softmax')(encoder_embedding_output)
    model = Model(inputs, preds)
    return model

In [ ]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

# Data loading and analysis

## Load data

In [ ]:
data_path = '../../dat/'


In [ ]:
data_file = 'imdb/labeledTrainData.tsv'
data_train = pd.read_csv(os.path.join(data_path, data_file), sep='\t')
print(data_train.shape)
data_train.head()

In [ ]:
data_train = data_train[:1000]

In [ ]:
data_file = 'imdb/testData.tsv'
data_test = pd.read_csv(os.path.join(data_path, data_file), sep='\t')
print(data_test.shape)
data_test.head()

In [ ]:
data_test = data_test[:1000]

In [ ]:
data_train.review

In [ ]:
all_texts = data_train.review  + data_test.review

### Histogram of lenghts

In [ ]:
#import nltk
#nltk.download('punkt')

In [ ]:

chars_per_words_lengths = []
words_per_sents_lengths = []
sents_per_docs_lengths = []

# Chars per word should be on all text

for text in all_texts:
    
    sents = sent_tokenize(clean_str(BeautifulSoup(text).get_text()))
    sents_per_docs_lengths.append(len(sents))
    for sent in sents:       
    
        words = word_tokenize(text)
        words_per_sents_lengths.append(len(words))
        for word in words:
            chars_per_words_lengths.append(len(word))

    

In [ ]:
h_s = plt.hist(sents_per_docs_lengths, bins=500)

In [ ]:
np.mean(sents_per_docs_lengths)

In [ ]:
np.max(sents_per_docs_lengths)

In [ ]:
np.std(sents_per_docs_lengths)

In [ ]:
h_w = plt.hist(words_per_sents_lengths, bins=500)

In [ ]:
np.mean(words_per_sents_lengths)

In [ ]:
np.max(words_per_sents_lengths)

In [ ]:
np.std(words_per_sents_lengths)

In [ ]:
h_c = plt.hist(chars_per_words_lengths, bins=500)

In [ ]:
np.mean(chars_per_words_lengths)

In [ ]:
np.max(chars_per_words_lengths)

In [ ]:
np.std(chars_per_words_lengths)

In [ ]:
all_texts = list(all_texts.apply(BeautifulSoup).apply(BeautifulSoup.get_text).apply(clean_str))


In [ ]:
train_texts = list(data_train.review.apply(BeautifulSoup).apply(BeautifulSoup.get_text).apply(clean_str))
test_texts = list(data_test.review.apply(BeautifulSoup).apply(BeautifulSoup.get_text).apply(clean_str))

## Build char vocab (all text)

In [ ]:

vocab_to_int, int_to_vocab = build_chars_vocab(all_texts)
#np.savez('vocab_char-{}'.format(max_sent_len), vocab_to_int=vocab_to_int, int_to_vocab=int_to_vocab, max_sent_len=max_sent_len, min_sent_len=min_sent_len )
char2int = vocab_to_int
int2char = int_to_vocab

In [ ]:
input_characters = sorted(list(vocab_to_int))
target_characters = sorted(list(vocab_to_int))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in all_texts])

In [ ]:
print('Number of samples:', len(all_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)


In [ ]:
char2int

In [ ]:
int2char

In [ ]:
len(int_to_vocab)

# Train review model

# Load documents data

In [ ]:
'''
MAX_SENTS_PER_DOC = int(np.mean(sents_per_docs_lengths)) + 1
MAX_WORDS_PER_SENT = int(np.mean(words_per_sents_lengths)) + 1
MAX_CHARS_PER_WORD = int(np.mean(chars_per_words_lengths)) + 1
'''
MAX_SENTS_PER_DOC = 10
MAX_WORDS_PER_SENT = 40
MAX_CHARS_PER_WORD = 20
print('MAX_SENTS_PER_DOC = ' + str(MAX_SENTS_PER_DOC) + '\n')
print('MAX_WORDS_PER_SENT = ' + str(MAX_WORDS_PER_SENT) + '\n')
print('MAX_CHARS_PER_WORD = ' + str(MAX_CHARS_PER_WORD) + '\n')

NUM_CLASSES = 2

EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.2

# Vectorize documents data

In [ ]:
train_input_data, train_targets = vectorize_sentences_data(input_texts=train_texts, 
                                                               target_labels=list(data_train.sentiment), 
                                                               max_sents_per_doc=MAX_SENTS_PER_DOC, 
                                                               max_words_per_sent=MAX_WORDS_PER_SENT, 
                                                               max_chars_per_word=MAX_CHARS_PER_WORD, 
                                                               num_classes=NUM_CLASSES, 
                                                               char2int=char2int)



In [ ]:
test_input_data, _ = vectorize_sentences_data(input_texts=test_texts, 
                                               target_labels=None, 
                                               max_sents_per_doc=MAX_SENTS_PER_DOC, 
                                               max_words_per_sent=MAX_WORDS_PER_SENT, 
                                               max_chars_per_word=MAX_CHARS_PER_WORD, 
                                               num_classes=NUM_CLASSES, 
                                               char2int=char2int)

In [ ]:
train_input_data.shape

In [ ]:
len(train_input_data)

In [ ]:
train_targets.shape

In [ ]:
test_input_data.shape

# Build model

In [ ]:
char2word_latent_dim = 512
word2sent_latent_dim = 256
sent2doc_latent_dim = 128
char_vocab_size = len(char2int)

#MAX_SENTS_PER_DOC = 11
#MAX_WORDS_PER_SENT = 24
#MAX_CHARS_PER_WORD = 5
#_, _, _, encoder_word_embedding_model = build_chars2word_model(num_encoder_tokens=char_vocab_size, latent_dim=chars2word_latent_dim)
encoder_word_embedding_model = build_chars2word_model_simple_BiLSTM(num_encoder_tokens=char_vocab_size, latent_dim=char2word_latent_dim)
print(encoder_word_embedding_model.summary())
'''
_, _, _, encoder_sentence_embedding_model = build_words2sent_model(encoder_word_embedding_model, 
                                                                   max_words_seq_len=MAX_WORDS_PER_SENT, 
                                                                   max_char_seq_len=MAX_CHARS_PER_WORD,
                                                                   latent_dim=words2sent_latent_dim)
'''
encoder_sentence_embedding_model = build_words2sent_model_simple_BiLSTM(encoder_word_embedding_model, 
                                                                   max_words_seq_len=MAX_WORDS_PER_SENT, 
                                                                   max_char_seq_len=MAX_CHARS_PER_WORD, 
                                                                   latent_dim=word2sent_latent_dim)
print(encoder_sentence_embedding_model.summary())

encoder_document_embedding_model = build_sent2doc_model(encoder_sentence_embedding_model, 
                                                 max_sents_seq_len=MAX_SENTS_PER_DOC, 
                                                 max_words_seq_len=MAX_WORDS_PER_SENT, 
                                                 max_char_seq_len=MAX_CHARS_PER_WORD, 
                                                 word2sent_latent_dim=word2sent_latent_dim,
                                                 sent2doc_latent_dim=sent2doc_latent_dim)
print(encoder_document_embedding_model.summary())
model = build_hier_senti_model(encoder_document_embedding_model=encoder_document_embedding_model,
                                max_sents_seq_len=MAX_SENTS_PER_DOC, 
                                max_words_seq_len=MAX_WORDS_PER_SENT, 
                                max_char_seq_len=MAX_CHARS_PER_WORD)
print(model.summary())





# Train model

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 1
lr = 0.01

model.compile(optimizer=optimizers.Adam(lr=lr), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

#filepath="weights-improvement-{epoch:02d}-{val_categorical_accuracy:.2f}.hdf5"
filepath="best_hier_senti_model-{}-{}.hdf5".format(MAX_SENTS_PER_DOC,MAX_WORDS_PER_SENT,MAX_CHARS_PER_WORD) # Save only the best model for inference step, as saving the epoch and metric might confuse the inference function which model to use
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list = [checkpoint, tbCallBack]
#callbacks_list = [checkpoint, tbCallBack, lrate]
model.fit(train_input_data, train_targets,
          #validation_data=(test_input_data, test_targets)
          batch_size=batch_size,
          epochs=epochs,
          callbacks=callbacks_list,
          validation_split=0.2,
          shuffle=True)

# Test model

In [ ]:
for i, rev in enumerate(test_texts):
    print(rev)
    sentiment = model.predict(test_input_data[i])
    print('Sentiment: ' + str(sentiment))
    